In [1]:
import json
import string
import random
import nltk
nltk.download('omw-1.4')
import numpy as num
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
import tensorflow as tensorF # A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import Sequential # Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")# required package for tokenization
nltk.download("wordnet")# word database

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
ourData = {"ourIntents": [

    {"tag": "greeting",
     "patterns": ["Hi", "Hello", "Hey"],
     "responses": ["Welcome to our online store! How can I assist you?", "Hello! How can I help you today?", "Hi there! How can we serve you?"],
    },

    {"tag": "product_inquiry",
     "patterns": ["Tell me about your products", "What do you sell?", "Product catalog"],
     "responses": ["We offer a wide range of products including electronics, clothing, accessories, and more. Feel free to explore our catalog on our website!"],
    },

    {"tag": "price_inquiry",
     "patterns": ["How much does it cost?", "What is the price?", "Tell me the price"],
     "responses": ["For the latest pricing information, please check our website or contact our customer support. Prices may vary based on promotions and availability."],
    },

    {"tag": "shipping_info",
     "patterns": ["Tell me about shipping", "How does shipping work?", "Delivery information"],
     "responses": ["We provide reliable and timely shipping services. Shipping fees and delivery times may vary based on your location. You can find more details during the checkout process."],
    },

    {"tag": "payment_info",
     "patterns": ["How can I pay?", "Payment options", "Accepted payment methods"],
     "responses": ["We accept various payment methods, including credit cards, PayPal, and more. During the checkout process, you can choose the payment option that suits you best."],
    },

    {"tag": "goodbye",
     "patterns": ["bye", "see you later", "goodbye"],
     "responses": ["Thank you for visiting our online store! If you have any more questions, feel free to ask. Goodbye!", "Goodbye! Have a great day!"],
    }

]}


In [3]:
lm = WordNetLemmatizer() #for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in ourData["ourIntents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)# tokenize the patterns
        newWords.extend(ournewTkns)# extends the tokens
        documentX.append(pattern)
        documentY.append(intent["tag"])


    if intent["tag"] not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [4]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)# coverting our data into an array afterv shuffling

x = num.array(list(trainingData[:, 0]))# first trainig phase
y = num.array(list(trainingData[:, 1]))# second training phase


In [5]:
iShape = (len(x[0]),)
oShape = len(y[0])
# parameter definition
ourNewModel = Sequential()
# In the case of a simple stack of layers, a Sequential model is appropriate

# Dense function adds an output layer
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
# The activation function in a neural network is in charge of converting the node's summed weighted input into activation of the node or output for the input in question
ourNewModel.add(Dropout(0.5))
# Dropout is used to enhance visual perception of input neurons
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "softmax"))
# below is a callable that returns the value to be used with no arguments
md = tensorF.keras.optimizers.Adam(learning_rate=0.005)
# Below line improves the numerical stability and pushes the computation of the probability distribution into the categorical crossentropy loss function.
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
# Output the model in summary
print(ourNewModel.summary())
# Whilst training your Nural Network, you have the option of making the output verbose or simple.
ourNewModel.fit(x, y, epochs=200, verbose=1)
# By epochs, we mean the number of times you repeat a training set.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4736      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 13382 (52.27 KB)
Trainable params: 13382 (52.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/200
1/1

In [6]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word) for word in newtkns]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["ourIntents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      ourResult = random.choice(i["responses"])
      break
  return ourResult

In [9]:
while True:
    newMessage = input("")

    # Tentukan kondisi untuk menghentikan loop
    if newMessage.lower() == "exit":
        break

    intents = Pclass(newMessage, newWords, ourClasses)
    ourResult = getRes(intents, ourData)
    print(ourResult)


hello
1/1 [==============================] - 0s 31ms/step
Welcome to our online store! How can I assist you?
what do you sell
1/1 [==============================] - 0s 21ms/step
We offer a wide range of products including electronics, clothing, accessories, and more. Feel free to explore our catalog on our website!
how much does it cost
1/1 [==============================] - 0s 29ms/step
For the latest pricing information, please check our website or contact our customer support. Prices may vary based on promotions and availability.
tell me about shipping
1/1 [==============================] - 0s 21ms/step
We provide reliable and timely shipping services. Shipping fees and delivery times may vary based on your location. You can find more details during the checkout process.
how can i pay
1/1 [==============================] - 0s 19ms/step
We accept various payment methods, including credit cards, PayPal, and more. During the checkout process, you can choose the payment option that suit